# Batting Analysis of The World Cup 2019
> Inspired by some graphics made by [@SAAdvantage](https://twitter.com/SAAdvantage)

In [1]:
# Imports
import requests
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import re

Define the URL for the summary page that shows the overall batting stats for the top 50 highest scoring batsman at The World Cup 2019.

In [3]:
url = 'http://stats.espncricinfo.com/ci/engine/records/batting/most_runs_career.html?id=12357;type=tournament'
soup = BeautifulSoup(requests.get(url).text, features="html.parser")

In [4]:
for caption in soup.find_all('caption'):
    if caption.get_text() == 'Most runs':
        main_table = caption.find_parent(
            'table', {'class': 'engineTable'})

In [7]:
columns = [header.get_text() for header in main_table.find('thead').find_all('tr')[0].find_all('th')]
rows = []

for innings in [row for row in main_table.find('tbody').find_all('tr')]:
    rows.append([stat.get_text() for stat in innings.find_all('td')])
    
df = pd.DataFrame(rows, columns=columns).apply(pd.to_numeric, errors='ignore')

In [8]:
df

,Player,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,0,4s,6s
0,RG Sharma (INDIA),9,9,1,648,140,81.00,659,98.33,5,1,0,67,14
1,DA Warner (AUS),10,10,1,647,166,71.88,724,89.36,3,3,0,66,8
2,Shakib Al Hasan (BDESH),8,8,1,606,124*,86.57,631,96.03,2,5,0,60,2
3,KS Williamson (NZ),10,9,2,578,148,82.57,771,74.96,2,2,0,50,3
4,JE Root (ENG),11,11,2,556,107,61.77,621,89.53,2,3,0,48,2
5,JM Bairstow (ENG),11,11,0,532,111,48.36,573,92.84,2,2,2,67,11
6,AJ Finch (AUS),10,10,0,507,153,50.70,497,102.01,2,3,1,47,18
7,Babar Azam (PAK),8,8,1,474,101*,67.71,540,87.77,1,3,0,50,2
8,BA Stokes (ENG),11,10,3,465,89,66.42,499,93.18,0,5,0,38,11
9,JJ Roy (ENG),8,7,0,443,153,63.28,384,115.36,1,4,0,51,12
